# Real Time Yoga Detection

### Importing Libraries

In [3]:
import mediapipe as mp 
import numpy as np 
import cv2 

import os  
import numpy as np 
import cv2 
from tensorflow.keras.utils import to_categorical

from keras.layers import Input, Dense 
from keras.models import Model

import cv2 
import numpy as np 
import mediapipe as mp 
from keras.models import load_model 

## Data collection function

In [4]:
def data_collection():
    def inFrame(lst):
        if lst[28].visibility > 0.6 and lst[27].visibility > 0.6 and lst[15].visibility>0.6 and lst[16].visibility>0.6:
            return True 
        return False
    
    cap = cv2.VideoCapture(1)

    name = input("Enter the name of the Asana : ")

    holistic = mp.solutions.pose
    holis = holistic.Pose()
    drawing = mp.solutions.drawing_utils

    X = []
    data_size = 0

    while True:
        lst = []

        _, frm = cap.read()

        frm = cv2.flip(frm, 1)

        res = holis.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))

        if res.pose_landmarks and inFrame(res.pose_landmarks.landmark):
            for i in res.pose_landmarks.landmark:
                lst.append(i.x - res.pose_landmarks.landmark[0].x)
                lst.append(i.y - res.pose_landmarks.landmark[0].y)

            X.append(lst)
            data_size = data_size+1

        else: 
            cv2.putText(frm, "Make Sure Full body visible", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)

        drawing.draw_landmarks(frm, res.pose_landmarks, holistic.POSE_CONNECTIONS)

        cv2.putText(frm, str(data_size), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)

        cv2.imshow("window", frm)

        if cv2.waitKey(1) == 27 or data_size>80:
            cv2.destroyAllWindows()
            cap.release()
            break


    np.save(f"{name}.npy", np.array(X))
    print(np.array(X).shape)

## Data training Function

In [5]:
def train():
    is_init = False
    size = -1

    label = []
    dictionary = {}
    c = 0

    for i in os.listdir():
        if i.split(".")[-1] == "npy" and not(i.split(".")[0] == "labels"):  
            if not(is_init):
                is_init = True 
                X = np.load(i)
                size = X.shape[0]
                y = np.array([i.split('.')[0]]*size).reshape(-1,1)
            else:
                X = np.concatenate((X, np.load(i)))
                y = np.concatenate((y, np.array([i.split('.')[0]]*size).reshape(-1,1)))

            label.append(i.split('.')[0])
            dictionary[i.split('.')[0]] = c  
            c = c+1


    for i in range(y.shape[0]):
        y[i, 0] = dictionary[y[i, 0]]
    y = np.array(y, dtype="int32")


    y = to_categorical(y)

    X_new = X.copy()
    y_new = y.copy()
    counter = 0 

    cnt = np.arange(X.shape[0])
    np.random.shuffle(cnt)

    for i in cnt: 
        X_new[counter] = X[i]
        y_new[counter] = y[i]
        counter = counter + 1


    ip = Input(shape=(X.shape[1]))

    m = Dense(128, activation="tanh")(ip)
    m = Dense(64, activation="tanh")(m)

    op = Dense(y.shape[1], activation="softmax")(m) 

    model = Model(inputs=ip, outputs=op)

    model.compile(optimizer='rmsprop', loss="categorical_crossentropy", metrics=['acc'])

    model.fit(X_new, y_new, epochs=80)


    model.save("model.h5")
    np.save("labels.npy", np.array(label))

## Interface function

In [6]:
def interface():
    def inFrame(lst):
        if lst[28].visibility > 0.6 and lst[27].visibility > 0.6 and lst[15].visibility>0.6 and lst[16].visibility>0.6:
            return True 
        return False

    model  = load_model("model.h5")
    label = np.load("labels.npy")



    holistic = mp.solutions.pose
    holis = holistic.Pose()
    drawing = mp.solutions.drawing_utils

    cap = cv2.VideoCapture(1)


    while True:
        lst = []

        _, frm = cap.read()

        window = np.zeros((940,940,3), dtype="uint8")

        frm = cv2.flip(frm, 1)

        res = holis.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))

        frm = cv2.blur(frm, (4,4))
        if res.pose_landmarks and inFrame(res.pose_landmarks.landmark):
            for i in res.pose_landmarks.landmark:
                lst.append(i.x - res.pose_landmarks.landmark[0].x)
                lst.append(i.y - res.pose_landmarks.landmark[0].y)

            lst = np.array(lst).reshape(1,-1)

            p = model.predict(lst)
            pred = label[np.argmax(p)]

            if p[0][np.argmax(p)] > 0.75:
                cv2.putText(window, pred , (180,180),cv2.FONT_ITALIC, 1.3, (0,255,0),2)

            else:
                cv2.putText(window, "Asana is either wrong not trained" , (100,180),cv2.FONT_ITALIC, 1.8, (0,0,255),3)

        else: 
            cv2.putText(frm, "Make Sure Full body visible", (100,450), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255),3)

            
        drawing.draw_landmarks(frm, res.pose_landmarks, holistic.POSE_CONNECTIONS,
                                connection_drawing_spec=drawing.DrawingSpec(color=(255,255,255), thickness=6 ),
                                landmark_drawing_spec=drawing.DrawingSpec(color=(0,0,255), circle_radius=3, thickness=3))


        window[420:900, 170:810, :] = cv2.resize(frm, (640, 480))

        cv2.imshow("window", window)

        if cv2.waitKey(1) == 27:
            cv2.destroyAllWindows()
            cap.release()
            break



In [13]:
function_input = input("Add - add data, Train - Train the data you have collected, Run - Run the Model")

if function_input == "Add":
    data_collection()
elif function_input == "Train":
    train()
else:
    interface

Epoch 1/80
13/13 [==============================] - 1s 3ms/step - loss: 1.2903 - acc: 0.4469
Epoch 2/80
13/13 [==============================] - 0s 5ms/step - loss: 0.8884 - acc: 0.8198
Epoch 3/80
13/13 [==============================] - 0s 5ms/step - loss: 0.6559 - acc: 0.9012
Epoch 4/80
13/13 [==============================] - 0s 4ms/step - loss: 0.4694 - acc: 0.9407
Epoch 5/80
13/13 [==============================] - 0s 5ms/step - loss: 0.3406 - acc: 0.9506
Epoch 6/80
13/13 [==============================] - 0s 5ms/step - loss: 0.2471 - acc: 0.9605
Epoch 7/80
13/13 [==============================] - 0s 5ms/step - loss: 0.1890 - acc: 0.9654
Epoch 8/80
13/13 [==============================] - 0s 5ms/step - loss: 0.1414 - acc: 0.9679
Epoch 9/80
13/13 [==============================] - 0s 4ms/step - loss: 0.1103 - acc: 0.9802
Epoch 10/80
13/13 [==============================] - 0s 5ms/step - loss: 0.0831 - acc: 0.9877A: 0s - loss: 0.0799 - acc: 0.988
Epoch 11/80
13/13 [=================